In [3]:
import pandas as pd
import re

import folium
import requests

In [1]:
def preprocess_station_name(station_name):
    station_name = re.sub(r'\([^)]*\)', '', station_name)
    
    if not station_name.endswith('역'):
        station_name += '역'
    
    return station_name

In [4]:
Congestion = pd.read_csv("../data/Seoul_Congestion_week_2022.csv", encoding='cp949', index_col=0)
Elevator = pd.read_csv("../data/elevator_area.csv", index_col=0)
Elevator = Elevator.rename(columns = {"면적" : "엘리베이터면적"})

Transfer = pd.read_csv("../data/서울시_환승역.csv", encoding='cp949')

Transfer['환승역명'] = Transfer['환승역명'].apply(preprocess_station_name)

Transfer.환승역명 = ["이수역" if i == "총신대입구역" else i for i in Transfer.환승역명 ]

Transfer = list(set(Transfer["환승역명"]))

In [8]:
Lease = pd.read_csv('../data/Lease.csv', encoding='euc-kr') 
Lease.columns = ['연번', '상가유형', '호선', '역명', '상가번호', '면적', '영업업종', '계약시작일자','계약종료일자', '월임대료', '사업진행단계']
Lease['역명'] = Lease['역명'].apply(preprocess_station_name)
Lease = Lease[Lease['상가유형']=='공실']

In [9]:
Lease = pd.concat([Lease.groupby("역명")["면적"].sum(), Lease.groupby("역명").count()["상가유형"]], axis = 1)
Lease = Lease.rename(columns = {"상가유형" : "공실수", "면적" : "공실면적"})
Lease = Lease.reset_index()
Lease.역명 = ["이수역" if i == "총신대입구역" else i for i in Lease.역명 ]

In [10]:
Population = pd.read_csv('../data/Population.csv', header=2, usecols=['동별(1)','동별(2)','소계','소계.1', '인구밀도 (명/㎢)'])
Population.columns = ['합계', '자치구' ,'세대', '인구', '인구밀도']
Transfer_name = pd.read_csv("../data/서울시_환승역.csv", encoding='cp949')[["호선", "환승역명"]]

In [ ]:
Transfer_name["자치구"] = 0
for i in range(len(Transfer_name)):
    line, subway = Transfer_name[["호선", "환승역명"]].iloc[i].values
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?query=" + subway + str(line) + "호선"
    headers = {"Authorization" : "KakaoAK 3206f186fc6467db13a17a82c6e3e668"}
    region = requests.get(url, headers = headers).json()["documents"][0]["address_name"].split()[1]
    Transfer_name.자치구[i] = region

Transfer_name = Transfer_name[["환승역명","자치구"]]
Transfer_name = Transfer_name.drop_duplicates()

C:\Users\Ryzen\AppData\Local\Temp\ipykernel_14876\3955305125.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Transfer_name.자치구[i] = region


In [ ]:
Population = Population[["자치구","세대","인구","인구밀도"]]

In [ ]:
Population = pd.merge(Transfer_name, Population, how = "left",on = "자치구")

In [ ]:
Population = Population.rename(columns = {"환승역명" : "역명"})

In [ ]:
Population['역명'] = Population['역명'].apply(preprocess_station_name)
Population.역명 = ["이수역" if i == "총신대입구역" else i for i in Population.역명 ]

In [ ]:
Elevator

In [134]:
Total_df = pd.merge(Congestion, Elevator, how = "inner", on = "역명")
Total_df = pd.merge(Total_df, Population, how = "inner", on = "역명")
Total_df = pd.merge(Total_df, Lease, how = "inner", on = "역명")
Total_df.index = Total_df["역명"]
Total_df = Total_df.loc[:,~Total_df.columns.isin(["역명"])] 
Total_df = Total_df.loc[:,~Total_df.columns.isin(["자치구"])]
Total_df = Total_df.rename(columns = {"역내개수" : "역내엘리베이터개수",
                           "역내용량최대" : "역내엘리베이터용량최대",
                           "역외개수" : "역외엘리베이터개수",
                           "역외용량최대" : "역외엘리베이터용량최대",
                           "총개수" : "엘리베이터총개수"})
Total_df

,5시30분,6시00분,6시30분,7시00분,7시30분,8시00분,8시30분,9시00분,9시30분,10시00분,...,역내엘리베이터개수,역내엘리베이터용량최대,역외엘리베이터개수,역외엘리베이터용량최대,엘리베이터총개수,세대,인구,인구밀도,공실면적,공실수
역명,,,,,,,,,,,,,,,,,,,,,
가락시장역,24.25,29.95,39.65,52.90,77.95,99.20,95.75,77.10,64.45,47.95,...,4.0,1000.0,2,1000,6.0,286123.0,661515.0,107605.0,30.44,1
공덕역,18.00,19.55,23.15,45.15,69.15,85.30,60.15,39.75,33.25,25.90,...,0.0,0.0,0,0,0.0,181192.0,375639.0,57462.0,22.00,1
군자역,54.85,41.60,45.35,66.40,85.05,99.45,76.15,76.65,69.05,48.85,...,0.0,0.0,0,0,0.0,170388.0,351959.0,56173.0,128.50,3
노원역,20.25,16.50,22.95,27.55,37.30,36.75,31.75,29.45,22.70,17.95,...,4.0,1000.0,4,1000,8.0,218367.0,504423.0,95384.0,40.00,1
당산역,21.45,28.90,29.55,41.30,71.95,96.60,69.95,50.30,38.10,33.05,...,4.0,750.0,2,750,6.0,190761.0,398290.0,66975.0,34.00,2
대림역,56.20,39.50,33.15,46.20,62.10,85.85,69.85,58.75,39.80,30.85,...,4.0,1000.0,4,1000,8.0,184467.0,416815.0,79150.0,22.00,1
대림역,56.20,39.50,33.15,46.20,62.10,85.85,69.85,58.75,39.80,30.85,...,4.0,1000.0,4,1000,8.0,190761.0,398290.0,66975.0,22.00,1
동대문역,38.25,30.25,32.80,49.45,64.50,94.20,83.50,56.25,62.65,39.70,...,6.0,1000.0,4,1000,10.0,72304.0,151127.0,28610.0,44.40,2
동묘앞역,13.25,16.40,19.85,27.55,44.60,62.55,56.60,47.95,41.95,37.20,...,10.0,1000.0,4,1000,14.0,72304.0,151127.0,28610.0,56.00,2
